In [ ]:
!nvidia-smi

Wed Nov 18 16:43:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
from sklearn.model_selection import train_test_split
from imutils import paths
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
import tensorflow as tf
import imutils
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/My Drive/African_wildlife/images/buffalo.zip' -d '/content/drive/My Drive/African_wildlife/images/buffalo'
!unzip '/content/drive/My Drive/African_wildlife/images/elephant.zip' -d '/content/drive/My Drive/African_wildlife/images/elephant'
!unzip '/content/drive/My Drive/African_wildlife/images/zebra.zip' -d '/content/drive/My Drive/African_wildlife/images/zebra'
!unzip '/content/drive/My Drive/African_wildlife/images/rhino.zip' -d '/content/drive/My Drive/African_wildlife/images/rhino'

In [ ]:
BASE_PATH = r"/content/drive/My Drive/African_wildlife"
IMAGES_PATH = os.path.sep.join([BASE_PATH, "images"])
ANNOTS_PATH = os.path.sep.join([BASE_PATH, "annotations"])

data = []
labels = []
imagePaths = []

# loop over all CSV files in the annotations directory
for csvPath in paths.list_files(ANNOTS_PATH, validExts=(".csv")):
	# load the contents of the current CSV annotations file
	rows = open(csvPath).read().strip().split("\n")

	# loop over the rows
	for row in rows:
		# break the row into the filename, bounding box coordinates,
		# and class label
		row = row.split(",")
		(filename, startX, startY, endX, endY, label) = row

		# derive the path to the input image, load the image (in
		# OpenCV format), and grab its dimensions
		imagePath = os.path.sep.join([IMAGES_PATH, label,
			filename])
		image = cv2.imread(imagePath)
		(h, w) = image.shape[:2]

		# load the image and preprocess it
		image = load_img(imagePath, target_size=(224, 224))
		image = img_to_array(image)

		# update our list of data, class labels, bounding boxes, and
		# image paths
		data.append(image)
		labels.append(label)
		imagePaths.append(imagePath)
# convert the data, class labels, and image paths to
# NumPy arrays, scaling the input pixel intensities from the range
# [0, 255] to [0, 1]
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)
imagePaths = np.array(imagePaths)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

print(labels)
# only there are only two labels in the dataset, then we need to use
# Keras/TensorFlow's utility function as well
if len(lb.classes_) == 2:
	labels = to_categorical(labels)

[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 ...
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]]


In [ ]:
#splitting data into training set and test set ; 95%
split = train_test_split(data, labels, imagePaths,
	test_size=0.90, random_state=38)
# unpack the data split
(testimages, trainimages) = split[:2]
(testlabels, trainlabels) = split[2:4]
(testpaths, trainpaths) = split[4:]

#creating validation set and training1 set from previously created training set; 90%
split2 =train_test_split(trainimages,trainlabels,test_size=0.9,random_state=7)

(valimages,train1images)=split2[:2] 
(vallabels,train1labels)=split2[2:4]

In [ ]:
print(train1images.shape)
print(valimages.shape)

(2296, 224, 224, 3)
(255, 224, 224, 3)


In [ ]:
class sequential_model:
    
    def __init__(self):
        self.num_of_blocks=0
        self.input_shape=[]
        self.model=tf.keras.models.Sequential()
        self.set_input_shape()
        self.add_input_layer(self.input_shape)
        self.add_convo_network()
        self.add_dense_layer()
        self.model.compile(loss='categorical_crossentropy', optimizer='sgd',metrics='accuracy')
    
    def set_input_shape(self):
        print("Enter shape of input layer")
        for i in range(3):
            ele=int(input())
            self.input_shape.append(ele)
            
    def add_convo_block(self,num_filters,activ,padd):
        self.model.add(keras.layers.Conv2D(num_filters, 3, activation=activ, padding=padd))
        self.model.add(keras.layers.MaxPooling2D(pool_size=2))
        self.model.add(keras.layers.Dropout(0.5))
        
        
    def add_convo_network(self):
        self.num_of_blocks=int(input("Enter the no of convolutional blocks you want in your model :"))
        for i in range(self.num_of_blocks):
            num_filters=int(input('Enter number of filters :'))
            activ=input('Enter activation for layer :')
            padd=input('Padding SAME or VALID :')
            self.add_convo_block(num_filters,activ,padd)

            
    def add_input_layer(self,lista):
        self.model.add(keras.layers.Input(shape=lista))
    
    def add_flatten_layer(self):
        self.model.add(keras.layers.Flatten())
    
    def add_dense_layer(self):
        x=input('Enter the number of output classes')
        self.model.add(keras.layers.Dense(x,activation='softmax'))
    

In [ ]:
model=sequential_model()

In [ ]:
from functools import partial
defaultconv=partial(keras.layers.Conv2D,
                    kernel_size=3,activation='relu', padding='VALID')#try changing padding
model=keras.models.Sequential([
    defaultconv(filters=64,kernel_size=5,input_shape=[224,224,3]),
    keras.layers.MaxPooling2D(pool_size=2),
    
    defaultconv(filters=128),
    keras.layers.Dropout(0.5),
    keras.layers.MaxPooling2D(pool_size=2),

    defaultconv(filters=256),
    keras.layers.Dropout(0.5),
    keras.layers.MaxPooling2D(pool_size=2),
    
    defaultconv(filters=512),
    keras.layers.MaxPooling2D(pool_size=2),

    keras.layers.Flatten(),
    
    keras.layers.Dense(units=4,activation='softmax'),
])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='sgd',metrics='accuracy')

In [ ]:
his=model.fit(train1images,train1labels,
               validation_data=(valimages,vallabels),
               epochs=25,
              batch_size=14,
              )

Epoch 1/25
164/164 [==============================] - 10s 60ms/step - loss: 1.1727 - accuracy: 0.4490 - val_loss: 1.2057 - val_accuracy: 0.5333
Epoch 2/25
164/164 [==============================] - 10s 58ms/step - loss: 0.9661 - accuracy: 0.5693 - val_loss: 1.2295 - val_accuracy: 0.4667
Epoch 3/25
164/164 [==============================] - 10s 59ms/step - loss: 0.9005 - accuracy: 0.6154 - val_loss: 1.0701 - val_accuracy: 0.5451
Epoch 4/25
164/164 [==============================] - 10s 59ms/step - loss: 0.8077 - accuracy: 0.6581 - val_loss: 1.0768 - val_accuracy: 0.6000
Epoch 5/25
164/164 [==============================] - 10s 59ms/step - loss: 0.7368 - accuracy: 0.7047 - val_loss: 0.9805 - val_accuracy: 0.6902
Epoch 6/25
164/164 [==============================] - 10s 60ms/step - loss: 0.6882 - accuracy: 0.7104 - val_loss: 0.9551 - val_accuracy: 0.6980
Epoch 7/25
164/164 [==============================] - 10s 60ms/step - loss: 0.6262 - accuracy: 0.7487 - val_loss: 0.9204 - val_accuracy:

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 220, 220, 64)      4864      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
dropout (Dropout)            (None, 108, 108, 128)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 256)       295168    
_________________________________________________________________
dropout_1 (Dropout)          (None, 52, 52, 256)       0

In [ ]:
model.save('/content/drive/MyDrive/African_wildlife/saved_model.h5')

In [ ]:
model_now=keras.models.load_model('/content/drive/MyDrive/model_save')
model_now.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 220, 220, 64)      4864      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
dropout (Dropout)            (None, 108, 108, 128)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 256)       295168    
_________________________________________________________________
dropout_1 (Dropout)          (None, 52, 52, 256)       0

In [ ]:
model_now.evaluate(testimages,testlabels)

5/5 [==============================] - 0s 51ms/step - loss: 0.3086 - accuracy: 0.9179


[0.30862945318222046, 0.9179104566574097]